In [2]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense
import numpy as np

data_path = r'F:\PycharmProjects\KDD\SafeDrug\SafeDrug-main\data\records_final_72features.pkl'
data = dill.load(open(data_path, 'rb'))
X = []
y = []
for patient in data:
    for admission in patient:
#         print(admission)
        drug = [0]*151
        disease = [0] * 1956
        for i in admission[0]:
            disease[i] = 1
        for i in admission[2]:
            drug[i] = 1
        X.append(disease + admission[3])
        y.append(drug)
X = np.array(X)
y = np.array(y)
# 划分数据集 9:1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)




In [ ]:
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

clf_multilabel = OneVsRestClassifier(LGBMClassifier(),n_jobs = 4)
clf_multilabel.fit(X_train,y_train)

In [ ]:
val_pred = clf_multilabel.predict(X_test)
train_pred= clf_multilabel.predict(X_train)



In [ ]:
def jaccard(y_gt, y_pred):
    score = []
    for b in range(y_gt.shape[0]):
        target = np.where(y_gt[b] == 1)[0]
        out_list = np.where(y_pred[b] == 1)[0]
        inter = set(out_list) & set(target)
        union = set(out_list) | set(target)
        jaccard_score = 0 if union == 0 else len(inter) / len(union)
        score.append(jaccard_score)
    return np.mean(score)

jaccard(y_test, val_pred)

In [ ]:
jaccard(y_train, train_pred)

In [ ]:
# 保存模型

import pickle
with open( r'C:\Users\pinkpigma\pinkpigma的同步盘\KDD研二上\jupyter\model\lightgbm.pkl', 'wb') as f:
    pickle.dump(clf_multilabel, f)


In [3]:
# 读取模型
with open(r'C:\Users\pinkpigma\pinkpigma的同步盘\KDD研二上\jupyter\model\lightgbm.pkl', 'rb') as f:
    clf_multilabel = pickle.load(f)


NameError: name 'pickle' is not defined

In [ ]:
import shap
for lightgbm_model in clf_multilabel.estimators_:
    shap_values = shap.TreeExplainer(lightgbm_model.booster_).shap_values(X_test[:10,:])
    print(len(shap_values[1]))
    exit()